In [14]:
import math
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from collections import namedtuple

%matplotlib inline

RANDOM_SEED = 100

# Present Valuing (Discount Factor)
# x = r, y = T-t
DF = lambda x, y: math.exp(-x * y)

# Initial setup
S0 = 100      # Stock value at t=0
E = 100       # Strike Price
r = 0.05      # annulized risk-free interest rate
vol = 0.2     # annulized volatility of the underlying
T = 1         # 1 year to expiry

BinaryOption = namedtuple("BinaryOption", ["name", "c_v", "p_v"])

## Theoretical Black Scholes Binary Call & Put Option Value

In [15]:
def bse_theoretical(S0, E, T, r, vol):
    
    # Theoretical Value
    d1 = (np.log(S0/E) + (r + 0.5*vol*vol)*T) / (vol* math.sqrt(T))

    d2 = d1 - vol*math.sqrt(T)
    
    print(d1, d2)
    
    # Using Asset or Nothing
    c_an = S0*norm.cdf(d1) - E*np.exp(-r*T)*norm.cdf(d2)
    
    p_an = -S0*norm.cdf(-d1) + E * DF(r, T) * norm.cdf(-d2)


    print("="*100, "\n")
    literal = "Asset or Nothing Values"
    print(literal)
    print("-"*(len(literal) + 2), "\n")
    print("Binary Call Option Value:  V(S, T):{:7.2f}".format(c_an))
    print("Binary Put  Option Value:  V(S, T):{:7.2f}".format(p_an))
    print("="*100, "\n")
    
    return BinaryOption("bse_theoretical", c_an, p_an)

option_bse = bse_theoretical(S0, E, T, r, vol)

0.35000000000000003 0.15000000000000002

Asset or Nothing Values
------------------------- 

Binary Call Option Value:  V(S, T):  10.45
Binary Put  Option Value:  V(S, T):   5.57



## One Giant Leap Binary Call Value

In [16]:
np.random.seed(RANDOM_SEED)

def bse_giant_leap(npaths=1E6, plot=True, output=True):
    global T, S0, E, r, vol, DF, option_bse
    
    N = int(npaths)
    
    # Generate some random numbers
    rn = np.random.standard_normal(size=N)

    
    s_t = S0*np.exp((r - 0.5*vol**2)*T + vol*rn*math.sqrt(T))
    
    if plot:
        fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(18, 6), dpi=70)

        s_t_norm = (s_t - np.mean(s_t)) / np.std(s_t)

        # Note for self: Pass density=true to normalize it such that integral over distribution = 1
        ax.hist(s_t_norm, bins=1000, density=True)

        ax.set_xlabel("Normalzed Stock price at time T")
        ax.set_ylabel("Frequency");
        ax.set_xlim([-4, 4])

    # Value of Binary Option
    # value = S0 * DF(r, T) * np.heaviside(s_t - strike, 0).mean()
    call_value = DF(r, T) * np.maximum(s_t - E, 0).mean()
    put_value = DF(r, T) * np.maximum(E - s_t, 0).mean()
    
    if output:
        print("="*100, "\n")
        print("Expected Value: {:7.5f}\n".format(np.mean(s_t))) # TODO: delete later
        literal = "Asset or Nothing Values"
        print(literal)
        print("-"*(len(literal) + 2), "\n")
        print("Binary Call Option Value:  V(S, T):{:7.2f}  | ({:5.2f})".format(call_value, option_bse.c_v))
        print("Binary Put  Option Value:  V(S, T):{:7.2f}  | ({:5.2f})".format(put_value, option_bse.p_v))
        print("="*100, "\n")
    
    return BinaryOption("euler1", call_value, put_value)

option_euler1 = bse_giant_leap(npaths=1E4, plot=False)


Expected Value: 105.18016

Asset or Nothing Values
------------------------- 

Binary Call Option Value:  V(S, T):  10.55  | (10.45)
Binary Put  Option Value:  V(S, T):   5.62  | ( 5.57)



## Euler-Maruyama + Milstein (optional)

In [17]:
np.random.seed(RANDOM_SEED)

def euler(s_grid, rn, dt, r, vol, nsteps=1):
    """
    Executes looping over square grid to generate a random walk.
    ------------------------------------------------------------
    param  sgrid:
    param     dt:
    param      r:
    param    vol:
    param nsteps:
    """
    global option_bse
    
    # Manually adjust flags
    mil_on = 1                # set t1 to turn Milstein component on
    exponent_form = False     # Exponential form of the random walk

    # iterate across columns
    if exponent_form:
        exponent = (r - 0.5*(vol**2))*dt + (vol * rn * math.sqrt(dt))
        con = np.exp(exponent)

    else:

        a = r * dt
        b = vol * rn * math.sqrt(dt)
        c = 0.5 * (vol**2)*(np.power(rn, 2) - 1) * dt

        con = 1 + a + b + mil_on * c

    # Begin Iteration
    for i in range(0, nsteps):

        s_grid[:, i+1] = s_grid[:, i] * con[:, i]

# Execute below
def euler_maruyama(npaths=1E4, nsteps=252, plot=False, output=True):
    """
    Main Function to execute
    """
    
    global T, S0, E, r, vol, DF
    # with N=100,000 and nsteps=252 this consumes up to 500 MB of RAM
    
    N = int(npaths)
    max_paths_to_plot = 1

    M = int(nsteps) + 1
    dt = T / int(nsteps)

    # random number drawn from a standard normal distribution
    # Note this is essentially Cell values, hence M-1, while stock is essentially Point Data in CFD terminology
    rn = np.random.standard_normal(size=N*(M-1)).reshape((N, (M-1)), order="C")

    # Stock prices for each path from 0 to N-1
    s_grid = np.zeros((N, M), order="C")

    # Initial values
    s_grid[:, 0] = S0

    # Calculate Stock Price by walking across passed in grid
    euler(s_grid, rn, dt, r, vol, nsteps=nsteps)

    # Extract results at end of random walk
    s_t = s_grid[:, -1]
    
    if plot:
        # Plot Random Walk Distribution at expiry
        fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(18, 6), dpi=70, sharey=True)
        ax1, ax2 = axes
        plt.tight_layout(pad=1.08, h_pad=None, w_pad=None)
        
        ax1.set_ylabel("S(t)");
        ax2.set_xlabel("Normalized Stock price at time T")

        # Going to use quantile for now to come up with some way of letter y limits scale with varying settings
        c = 5/100/100
        q1, q3 = np.quantile(s_t, (c, 1-c))  # Q1 and Q3 quantiles
        ax2.set_ylim([q1, q3])

        # Plot Histogram
        ax2.hist(s_t, bins=1000, density=True, orientation="horizontal")

        # Plot the paths - No. paths limited by max_paths_to_plot variable :TODO fix this horrible variable name
        x = np.arange(0, M, step=1)
        for i in range(N):

            ax1.plot(x, s_grid[i, :])

            if i > max_paths_to_plot: break
        
        ax1.plot(x, s_grid.mean(axis=0).T)
        ax1.plot(x, np.ones_like(x)*S0, lw=2, color="k")
        ax1.set_xlim([0, nsteps])

    # Calculate Binary Option Value
    call_value = DF(r, T) * np.maximum(s_t - E, 0).mean()
    put_value = DF(r, T) * np.maximum(E - s_t, 0).mean()
    
    if output:
        print("="*100, "\n")
        print("Expected Value: {:7.5f}\n".format(np.mean(s_t))) # TODO: delete later
        literal = "Asset or Nothing Values"
        print(literal)
        print("-"*(len(literal) + 2), "\n")
        print("Binary Call Option Value:  V(S, T):{:7.2f}  | ({:5.2f})".format(call_value, option_bse.c_v))
        print("Binary Put  Option Value:  V(S, T):{:7.2f}  | ({:5.2f})".format(put_value, option_bse.p_v))
        print("="*100, "\n")
    
    return BinaryOption("euler2", call_value, put_value)

option_euler2 = euler_maruyama(npaths=1E4, nsteps=252, plot=False, output=True)


Expected Value: 105.23160

Asset or Nothing Values
------------------------- 

Binary Call Option Value:  V(S, T):  10.48  | (10.45)
Binary Put  Option Value:  V(S, T):   5.50  | ( 5.57)



# Alternative coding method: Euler-Maruyama + Milstein (optional)
- Just looking for something more memory efficient when number of timesteps and number of paths increase

In [20]:
np.random.seed(RANDOM_SEED)

def euler_alternative(s_t0, s_t1, dt, r, vol, nsteps=1):
    
    mil_on = 1               # set t1 to turn Milstein component on
    
    dt_sqrt = math.sqrt(dt)  # calculate this only once rather than every loop
    n = s_t0.size
    
    # Begin iteration
    for i in range(nsteps):
        
        # Generate N(0, 1) random numbers 
        rn = np.random.standard_normal(n)
        
        # Calculate value at next time step
        s_t1[:] = s_t0[:] * (1 + (r * dt) + (vol * rn * dt_sqrt)
                             + (mil_on * 0.5 * (vol**2)*(np.power(rn, 2) - 1) * dt))  # Milstein component
        
        # Setup next time step initial value values
        s_t0[:] = s_t1[:]
    
    return s_t1

def euler_maruyama_alternative(npaths=1E4, nsteps=252, plot=False, output=True):
    """
    Main Function to execute
    
    param npaths:  No. paths to simulate
    param nsteps:  No. time steps
    """
    
    global T, S0, E, r, vol, DF, option_bse
    
    N = int(npaths)
    M = int(nsteps)
    dt = T / M 

    # S_t - Initial stock prices for each path
    s_t0 = np.ones(N, order="C") * S0

    # S_t+1 - stock  price at next time step
    s_t1 = np.zeros_like(s_t0)

    # Calculate Stock Price
    s_t = euler_alternative(s_t0, s_t1, dt, r, vol, nsteps=nsteps)
    
    if plot:
        fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12, 6), dpi=70)

        s_t_norm = (s_t - np.mean(s_t)) / np.std(s_t)

        ax.hist(s_t_norm, bins=int(math.modf(N/10)[1]), range=[-4, 4])
        ax.set_xlabel("Normalized Stock price at time T")
        ax.set_ylabel("Frequency");

    # Calculate Binary Option Value
    call_value = DF(r, T) * np.maximum(s_t - E, 0).mean()
    put_value = DF(r, T) * np.maximum(E - s_t, 0).mean()
    
    if output:
        print("="*100, "\n")
        print("Expected Value: {:7.5f}\n".format(np.mean(s_t))) # TODO: delete later
        literal = "Asset or Nothing Values"
        print(literal)
        print("-"*(len(literal) + 2), "\n")
        print("Binary Call Option Value:  V(S, T):{:7.2f}  | ({:5.2f})".format(call_value, option_bse.c_v))
        print("Binary Put  Option Value:  V(S, T):{:8.2f}  | ({:5.2f})".format(put_value, option_bse.p_v))
        print("="*100, "\n")
    
    return BinaryOption("euler3", call_value, put_value)

option_euler3 = euler_maruyama_alternative(npaths=1E4, nsteps=252, plot=False)


Expected Value: 105.21911

Asset or Nothing Values
------------------------- 

Binary Call Option Value:  V(S, T):  10.44  | (10.45)
Binary Put  Option Value:  V(S, T):    5.48  | ( 5.57)



In [6]:
%timeit -n 10 -r 10 euler_maruyama(npaths=1E4, nsteps=252, plot=False, output=False)

218 ms ± 3.99 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [8]:
%timeit -n 10 -r 10 euler_maruyama_alternative(npaths=1E4, nsteps=252, plot=False, output=False)

122 ms ± 798 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)


# Convergence 